### Yを予測するための効率的なトレーニングデータの抽出検討

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pubchempy
import random
 
from ord_schema import message_helpers, validations
from ord_schema.proto import dataset_pb2
from rdkit import rdBase, Chem, DataStructs
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import AllChem, Draw, rdMHFPFingerprint
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.AtomPairs import Pairs, Torsions

In [3]:
pb = "./practice/ord_dataset-00005539a1e04c809a9a78647bea649c.pb.gz" #Buchwald-Hartwig Reaction datasetの読み込み 
data = message_helpers.load_message(pb, dataset_pb2.Dataset) #生データであるjson型式のデータファイルとして読み取っている?
df = message_helpers.messages_to_dataframe(data.reactions, drop_constant_columns=False) #jsonからdataframeへ変換

#元dfからA+B→Yとなる部分だけ抜き出してDataFrameにする
df = df[[ \
    'inputs["aryl halide"].components[0].identifiers[0].value', \
    'inputs["amine"].components[0].identifiers[0].value', \
    'outcomes[0].products[0].identifiers[0].value' \
    ]] 

df.columns = list('ABY') #列ラベルをわかりやすく A,B,Yに変換

#SMILESからMOLオブジェクトを生成する関数を定義
def generate_mol(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol
    except:
        return None

# dfのすべてのA,B 要素に対してMolオブジェクトを生成する
sr_mol_A = df["A"].apply(lambda smiles: generate_mol(smiles))
sr_mol_B = df["B"].apply(lambda smiles: generate_mol(smiles))

print(sr_mol_A.describe())
print(sr_mol_B.describe())

buchwald_hardwig_smiles_df = df.drop_duplicates()
buchwald_hardwig_smiles_df.insert(0, ":", ":")
buchwald_hardwig_smiles_df.insert(2, "+", "+")
buchwald_hardwig_smiles_df.insert(4, "→", "→")
#buchwald_hardwig_smiles_df.insert(6, "\", "\")
#buchwald_hardwig_smiles_df

count                                               750
unique                                              750
top       <rdkit.Chem.rdchem.Mol object at 0x15c6b19a0>
freq                                                  1
Name: A, dtype: object
count                                               750
unique                                              750
top       <rdkit.Chem.rdchem.Mol object at 0x15c6cdb60>
freq                                                  1
Name: B, dtype: object


## Buchwald-Hartwig Reaction データセット の FingerPrint　と　類似性

タニモト係数

Tanimoto Coefficient = c / (a + b - c)

・aは分子Aのビット配列で1が立っている数

・bは分子Bのビット配列で1が立っている数

・cは分子AとBで共通に1が立っている数

In [4]:
# 以下の化合物をテストデータとしてデータセットの各化合物とタニモト係数を計算する
#reactant_A = "CC1=NN(C=C1NC2=NC=C(C(=C2)I)C(F)(F)F)C"
#reactant_B = "CONC(=O)C1=CC=CC=C1N"

#mol_A = random.choice(sr_mol_A)
#mol_B = random.choice(sr_mol_B)

#Fingerprintから対象のテスト分子に対する関数を定義 
def calc_fps_tanimoto(test_fps, fps):
    fps_tnmt_coeffi = DataStructs.TanimotoSimilarity(test_fps, fps)
    return fps_tnmt_coeffi

1. MACCS Keys

    AllChem.GetMACCSKeysFingerprint(mol)

    166の部分構造について部分構造を有する場合は1が無い場合は0が格納される

In [5]:
#MACCAS Keys Fingerprintを生成する関数を定義 
def generate_maccs_fps(mol):
    maccs_fps = AllChem.GetMACCSKeysFingerprint(mol)
    return maccs_fps

#sr_molの各要素に対して maccs_fps を生成
sr_maccs_fps_A = sr_mol_A.apply(lambda mol: generate_maccs_fps(mol))
sr_maccs_fps_B = sr_mol_B.apply(lambda mol: generate_maccs_fps(mol))

#テスト分子ABのfpsをランダムに抜き出す
test_A_maccs_fps = random.choice(sr_maccs_fps_A)
test_B_maccs_fps = random.choice(sr_maccs_fps_B)

#sr_maccs_fpsの各要素に対して タニモト係数 を計算
sr_maccs_fps_tnmt_A = sr_maccs_fps_A.apply(lambda fps: calc_fps_tanimoto(test_A_maccs_fps, fps) \
                                           if fps != test_A_maccs_fps else None).dropna()
sr_maccs_fps_tnmt_B = sr_maccs_fps_B.apply(lambda fps: calc_fps_tanimoto(test_B_maccs_fps, fps)  \
                                           if fps != test_B_maccs_fps else None).dropna()

print(sr_maccs_fps_tnmt_A.describe()) 
print(sr_maccs_fps_tnmt_B.describe())



count    737.000000
mean       0.298377
std        0.126907
min        0.100000
25%        0.214286
50%        0.259259
75%        0.347826
max        0.863636
Name: A, dtype: float64
count    707.000000
mean       0.371793
std        0.177869
min        0.078431
25%        0.233259
50%        0.333333
75%        0.469388
max        0.937500
Name: B, dtype: float64


2. Topologicalフィンガープリント (RDKitフィンガープリント)

    Chem.RDKFingerprint(mol)

    一定の結合数に相当する原子と結合種類を格納(?)

In [6]:
#Topological(Rdkit) Fingerprintを生成する関数を定義 
def generate_rdkit_fps(mol):
    rdkit_fps = Chem.RDKFingerprint(mol)
    return rdkit_fps

#df_molの各要素に対して rdkit_fps を生成
sr_rdkit_fps_A = sr_mol_A.apply(lambda mol: generate_rdkit_fps(mol))
sr_rdkit_fps_B = sr_mol_B.apply(lambda mol: generate_rdkit_fps(mol))
#rdkit_fps_df

#テスト分子ABのfpsをランダムに抜き出す
test_A_rdkit_fps = random.choice(sr_rdkit_fps_A)
test_B_rdkit_fps = random.choice(sr_rdkit_fps_B)

#sr_rdkit_fpsの各要素に対して タニモト係数 を計算
sr_rdkit_fps_tnmt_A = sr_rdkit_fps_A.apply(lambda fps: calc_fps_tanimoto(test_A_rdkit_fps, fps) \
                                           if fps != test_A_rdkit_fps else None).dropna()
sr_rdkit_fps_tnmt_B = sr_rdkit_fps_B.apply(lambda fps: calc_fps_tanimoto(test_B_rdkit_fps, fps) \
                                           if fps != test_B_rdkit_fps else None).dropna()

print(sr_rdkit_fps_tnmt_A.describe()) 
print(sr_rdkit_fps_tnmt_B.describe())

count    748.000000
mean       0.142669
std        0.152393
min        0.033898
25%        0.059459
50%        0.077796
75%        0.127889
max        0.833333
Name: A, dtype: float64
count    736.000000
mean       0.074561
std        0.110884
min        0.000000
25%        0.013239
50%        0.023310
75%        0.095890
max        0.578947
Name: B, dtype: float64


3. Morganフィンガープリント (Circularフィンガープリント)

    AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)

    基準となる原子からある距離にある部分構造を数え上げていく

In [7]:
#Morgan Fingerprintを生成する関数を定義 
def generate_morgan_fps(mol, radius, nBits):
    morgan_fps = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
    return morgan_fps

#df_molの各要素に対して maccs_fps を生成
sr_morgan_fps_A = sr_mol_A.apply(lambda mol: generate_morgan_fps(mol, 2, 2048))
sr_morgan_fps_B = sr_mol_B.apply(lambda mol: generate_morgan_fps(mol, 2, 2048))
#morgan_fps_df

#テスト分子ABのfpsをランダムに抜き出す
test_A_morgan_fps = random.choice(sr_morgan_fps_A)
test_B_morgan_fps = random.choice(sr_morgan_fps_B)

#sr_morgan_fpsの各要素に対して タニモト係数 を計
sr_morgan_fps_tnmt_A = sr_morgan_fps_A.apply(lambda fps: calc_fps_tanimoto(test_A_morgan_fps, fps) \
                                           if fps != test_A_morgan_fps else None).dropna()
sr_morgan_fps_tnmt_B = sr_morgan_fps_B.apply(lambda fps: calc_fps_tanimoto(test_B_morgan_fps, fps) \
                                           if fps != test_B_morgan_fps else None).dropna()

print(sr_morgan_fps_tnmt_A.describe()) 
print(sr_morgan_fps_tnmt_B.describe())

count    749.000000
mean       0.136355
std        0.054417
min        0.042254
25%        0.098361
50%        0.131579
75%        0.160714
max        0.307692
Name: A, dtype: float64
count    734.000000
mean       0.070837
std        0.043092
min        0.000000
25%        0.033333
50%        0.071429
75%        0.094340
max        0.428571
Name: B, dtype: float64
